In [1]:
%load_ext autoreload
%autoreload 2

import cProfile
import numpy as np
import tensorflow as tf

from SpectralLayer import*
from utilsSimpleConv2D import*
from spectralconvolutions import *

from tensorflow.keras.layers import Layer
from typing import Tuple,List,Any,Dict

In [5]:
def build_model(model=None,batch_size=None, epochs=20):
    accuracy=list()
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1,validation_data=(x_test, y_test))
    accuracy.append(model.evaluate(x_test,y_test,batch_size=32,verbose="auto"))
    return accuracy

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test=x_train.reshape(-1,28,28,1), x_test.reshape(-1,28,28,1)

x_train, x_test = x_train / 255.0, x_test / 255.0

flat_train = np.reshape(x_train, [x_train.shape[0], 28 * 28])
flat_test = np.reshape(x_test, [x_test.shape[0], 28 * 28])

In [3]:
epochs = 20
batch_size=200
accuracy=list()

spectral_config={
    'is_base_trainable' : False,
    'is_diag_start_trainable' : False,
    'is_diag_end_trainable' : True,
}

In [4]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(28,28,1)))

model.add(PaddingJacobiens(kernel_size=3, strides=1, padding="VALID"))
model.add(SpecConv2D(filters=1, kernel_size=2,use_lambda_in=True,use_bias=False, activation="relu"))
model.add(tf.keras.layers.MaxPooling2D((2,2)))     

model.add(tf.keras.layers.Flatten())     

model.add(Spectral(100, **spectral_config,use_bias=False, activation=None))
model.add(Spectral(10, **spectral_config,use_bias=False, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 padding_jacobiens (Padding  (None, 78, 78, 1)         0         
 Jacobiens)                                                      
                                                                 
 spec_conv2d (SpecConv2D)    (None, 39, 39, 1)         6084      
                                                                 
 max_pooling2d (MaxPooling2  (None, 19, 19, 1)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 361)               0         
                                                                 
 module_wrapper (ModuleWrap  (None, 100)               100       
 per)                                                            
                                                        

In [6]:
build_model(model,batch_size=batch_size)

Epoch 1/20
300/300 [==============================] - 1716s 6s/step - loss: 1.8826 - accuracy: 0.5963 - val_loss: 0.6910 - val_accuracy: 0.7922
Epoch 2/20
300/300 [==============================] - 1235s 4s/step - loss: 0.6026 - accuracy: 0.8176 - val_loss: 0.5079 - val_accuracy: 0.8510
Epoch 3/20
300/300 [==============================] - 1223s 4s/step - loss: 0.4725 - accuracy: 0.8600 - val_loss: 0.4293 - val_accuracy: 0.8764
Epoch 4/20
300/300 [==============================] - 1120s 4s/step - loss: 0.4075 - accuracy: 0.8793 - val_loss: 0.3842 - val_accuracy: 0.8900
Epoch 5/20
300/300 [==============================] - 1120s 4s/step - loss: 0.3684 - accuracy: 0.8913 - val_loss: 0.3557 - val_accuracy: 0.8960
Epoch 6/20
300/300 [==============================] - 1110s 4s/step - loss: 0.3410 - accuracy: 0.8991 - val_loss: 0.3277 - val_accuracy: 0.9066
Epoch 7/20
300/300 [==============================] - 1114s 4s/step - loss: 0.3199 - accuracy: 0.9055 - val_loss: 0.3130 - val_accuracy:

[[0.25151970982551575, 0.9253000020980835]]